<a href="https://colab.research.google.com/github/Ashishkhurana01/Hands-On-LLM/blob/main/ch03_LookingInsideLLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Looking Inside Transformer LLMs

In [5]:
%%capture
!pip install transformers>=4.41.2 accelerate>=0.31.0